# Lignin Oligomer Molecular Graph Analysis

This notebook performs comprehensive analysis of lignin oligomers from the LigninStructs_3.json dataset.

## Analysis Overview:
1. **Data Loading**: Load lignin oligomer structures from JSON file
2. **Atom-level Visualization**: Visualize molecular graphs for all oligomers
3. **Atom-level Network Statistics**: Calculate degree, clustering, and connectivity metrics
4. **Aggregated Statistics**: Summarize network properties across all oligomers
5. **Monomer-level Analysis**: Analyze using bond counts as proxy for inter-monomer connections

**Requirements**: pandas, matplotlib, NetworkX, RDKit, numpy

**Dataset**: All oligomers have DP (Degree of Polymerization) = 3

## 1. Setup and Imports

In [ ]:
# Essential imports
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# RDKit imports for molecular structure handling
from rdkit import Chem
from rdkit.Chem import Draw, Descriptors, rdMolDescriptors
from rdkit.Chem.Draw import rdMolDraw2D

# Set plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("All libraries imported successfully!")
print(f"RDKit version: {Chem.rdBase.rdkitVersion}")
print(f"NetworkX version: {nx.__version__}")
print(f"Pandas version: {pd.__version__}")

## 2. Data Loading from LigninStructs_3.json

In [ ]:
# Load the lignin structures dataset
with open('LigninStructs_3.json', 'r') as f:
    data = json.load(f)

print("Dataset Overview:")
print(f"Monomer Type: {data['monoType']}")
print(f"Degree of Polymerization (DP): {data['dp']}")
print(f"Number of Structures: {data['nunber_of_structs']}")
print(f"Average Molecular Weight: {data['avgMolWt']:.1f} g/mol")
print(f"S/G Ratio: {data['s_G_Ratio']:.2f}")
print(f"Number of Linear Chains: {data['noOflinearChains']}")
print(f"Number of Branched Chains: {data['noOfBranchedChains']}")

# Extract lignin chains for analysis
lignin_chains = data['ligninchains']
print(f"\nTotal oligomers to analyze: {len(lignin_chains)}")

# Display basic information about each oligomer
print("\nOligomer Details:")
for i, chain in enumerate(lignin_chains):
    print(f"  {i+1}. ID: {chain['lg_id']}, MW: {chain['molWeight']} g/mol, Bonds: {chain['bndCnts']}")

## 3. Molecular Structure Processing

Convert SMILES strings to RDKit molecule objects and create NetworkX graphs for analysis.

In [ ]:
def smiles_to_networkx(smiles):
    """
    Convert SMILES string to NetworkX graph representing the molecular structure.
    
    Args:
        smiles (str): SMILES representation of the molecule
        
    Returns:
        nx.Graph: NetworkX graph with atoms as nodes and bonds as edges
    """
    # Parse SMILES string
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Failed to parse SMILES: {smiles}")
        return None
    
    # Add explicit hydrogens for complete graph
    mol = Chem.AddHs(mol)
    
    # Create NetworkX graph
    G = nx.Graph()
    
    # Add atoms as nodes with properties
    for atom in mol.GetAtoms():
        G.add_node(atom.GetIdx(), 
                  symbol=atom.GetSymbol(),
                  atomic_num=atom.GetAtomicNum(),
                  degree=atom.GetDegree(),
                  hybridization=str(atom.GetHybridization()),
                  is_aromatic=atom.GetIsAromatic())
    
    # Add bonds as edges with properties
    for bond in mol.GetBonds():
        G.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(),
                  bond_type=str(bond.GetBondType()),
                  is_aromatic=bond.GetIsAromatic(),
                  bond_order=bond.GetBondTypeAsDouble())
    
    return G, mol

# Process all oligomers
oligomer_data = []
molecular_graphs = []
rdkit_mols = []

print("Processing SMILES strings...")
for i, chain in enumerate(lignin_chains):
    result = smiles_to_networkx(chain['smilestring'])
    if result is not None:
        G, mol = result
        oligomer_data.append({
            'id': chain['lg_id'],
            'smiles': chain['smilestring'],
            'mol_weight': chain['molWeight'],
            'bond_counts': chain['bndCnts'],
            'graph': G,
            'rdkit_mol': mol
        })
        molecular_graphs.append(G)
        rdkit_mols.append(mol)
        print(f"  ✓ Processed {chain['lg_id']}: {G.number_of_nodes()} atoms, {G.number_of_edges()} bonds")
    else:
        print(f"  ✗ Failed to process {chain['lg_id']}")

print(f"\nSuccessfully processed {len(oligomer_data)} oligomers")

## 4. Atom-level Molecular Graph Visualization

Visualize the molecular structures using both RDKit's chemical drawing and NetworkX graph layouts.

In [ ]:
# Chemical structure visualization using RDKit
print("Molecular Structure Visualizations (Chemical Drawings)")
print("=" * 55)

# Create a figure with subplots for all structures
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, oligo in enumerate(oligomer_data):
    if i < 5:  # Display first 5 structures
        # Generate 2D coordinates
        mol_copy = Chem.Mol(oligo['rdkit_mol'])
        # Remove explicit hydrogens for cleaner visualization
        mol_no_h = Chem.RemoveHs(mol_copy)
        
        # Draw molecule
        img = Draw.MolToImage(mol_no_h, size=(400, 300))
        
        # Display in subplot
        axes[i].imshow(img)
        axes[i].set_title(f"{oligo['id']}\nMW: {oligo['mol_weight']} g/mol")
        axes[i].axis('off')

# Remove empty subplot
if len(oligomer_data) < 6:
    axes[5].axis('off')

plt.tight_layout()
plt.suptitle('Lignin Oligomer Chemical Structures', fontsize=16, y=0.98)
plt.show()

print(f"\nDisplayed chemical structures for {min(5, len(oligomer_data))} oligomers")

In [ ]:
# Network graph visualization using NetworkX
print("Molecular Graph Network Visualizations (Atom-level)")
print("=" * 52)

def plot_molecular_graph(G, title, ax, pos_seed=42):
    """
    Plot a molecular graph using NetworkX with atoms colored by element type.
    """
    # Define colors for different elements
    element_colors = {
        'C': '#404040',  # Carbon - dark gray
        'O': '#FF0000',  # Oxygen - red  
        'H': '#FFFFFF',  # Hydrogen - white
        'N': '#0000FF',  # Nitrogen - blue
        'S': '#FFFF00'   # Sulfur - yellow
    }
    
    # Get node colors based on atom symbols
    node_colors = [element_colors.get(G.nodes[node]['symbol'], '#808080') for node in G.nodes()]
    
    # Get node sizes based on atomic number (larger for heavier atoms)
    node_sizes = [max(50, G.nodes[node]['atomic_num'] * 15) for node in G.nodes()]
    
    # Use spring layout for better visualization
    pos = nx.spring_layout(G, seed=pos_seed, k=1, iterations=50)
    
    # Draw the graph
    nx.draw(G, pos, ax=ax,
            node_color=node_colors,
            node_size=node_sizes,
            edge_color='gray',
            width=1.5,
            with_labels=False)
    
    # Add element labels for non-hydrogen atoms
    non_h_nodes = {node: data for node, data in G.nodes(data=True) if data['symbol'] != 'H'}
    if len(non_h_nodes) < 50:  # Only label if not too crowded
        labels = {node: data['symbol'] for node, data in non_h_nodes.items()}
        nx.draw_networkx_labels(G, pos, labels, ax=ax, font_size=8, font_color='white')
    
    ax.set_title(title)
    return pos

# Create network visualizations
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, oligo in enumerate(oligomer_data):
    if i < 5:  # Display first 5 graphs
        title = f"{oligo['id']}\n{oligo['graph'].number_of_nodes()} atoms, {oligo['graph'].number_of_edges()} bonds"
        plot_molecular_graph(oligo['graph'], title, axes[i], pos_seed=42+i)

# Remove empty subplot and add legend
if len(oligomer_data) < 6:
    axes[5].axis('off')
    # Add legend for element colors
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='#404040', label='Carbon (C)'),
        Patch(facecolor='#FF0000', label='Oxygen (O)'),
        Patch(facecolor='#FFFFFF', edgecolor='black', label='Hydrogen (H)')
    ]
    axes[5].legend(handles=legend_elements, loc='center', fontsize=12)
    axes[5].set_title('Element Color Legend')

plt.tight_layout()
plt.suptitle('Lignin Oligomer Molecular Graphs (Atom-level Networks)', fontsize=16, y=0.98)
plt.show()

print(f"\nDisplayed network graphs for {min(5, len(oligomer_data))} oligomers")

## 5. Atom-level Network Statistics

Calculate comprehensive network statistics for each oligomer including degree distribution, clustering coefficients, and connectivity metrics.

In [ ]:
def calculate_atom_network_stats(G, oligomer_id):
    """
    Calculate comprehensive network statistics for a molecular graph.
    
    Args:
        G (nx.Graph): Molecular graph
        oligomer_id (str): Identifier for the oligomer
        
    Returns:
        dict: Dictionary containing various network statistics
    """
    stats = {'oligomer_id': oligomer_id}
    
    # Basic graph properties
    stats['num_atoms'] = G.number_of_nodes()
    stats['num_bonds'] = G.number_of_edges()
    stats['density'] = nx.density(G)
    
    # Degree statistics
    degrees = [d for n, d in G.degree()]
    stats['avg_degree'] = np.mean(degrees)
    stats['max_degree'] = np.max(degrees)
    stats['min_degree'] = np.min(degrees)
    stats['degree_std'] = np.std(degrees)
    
    # Clustering and connectivity
    stats['avg_clustering'] = nx.average_clustering(G)
    stats['is_connected'] = nx.is_connected(G)
    
    if nx.is_connected(G):
        stats['diameter'] = nx.diameter(G)
        stats['avg_path_length'] = nx.average_shortest_path_length(G)
        stats['radius'] = nx.radius(G)
        stats['wiener_index'] = nx.wiener_index(G)
    else:
        # For disconnected graphs, calculate for largest component
        largest_cc = max(nx.connected_components(G), key=len)
        subgraph = G.subgraph(largest_cc)
        stats['diameter'] = nx.diameter(subgraph)
        stats['avg_path_length'] = nx.average_shortest_path_length(subgraph)
        stats['radius'] = nx.radius(subgraph)
        stats['wiener_index'] = nx.wiener_index(subgraph)
        stats['num_components'] = nx.number_connected_components(G)
    
    # Element composition
    element_counts = Counter([G.nodes[node]['symbol'] for node in G.nodes()])
    stats['carbon_count'] = element_counts.get('C', 0)
    stats['oxygen_count'] = element_counts.get('O', 0)
    stats['hydrogen_count'] = element_counts.get('H', 0)
    
    # Aromatic information
    aromatic_atoms = sum(1 for node in G.nodes() if G.nodes[node]['is_aromatic'])
    aromatic_bonds = sum(1 for u, v in G.edges() if G.edges[u, v]['is_aromatic'])
    stats['aromatic_atoms'] = aromatic_atoms
    stats['aromatic_bonds'] = aromatic_bonds
    stats['aromaticity_ratio'] = aromatic_atoms / stats['num_atoms'] if stats['num_atoms'] > 0 else 0
    
    return stats

# Calculate statistics for all oligomers
print("Calculating atom-level network statistics...")
atom_stats = []
for oligo in oligomer_data:
    stats = calculate_atom_network_stats(oligo['graph'], oligo['id'])
    atom_stats.append(stats)
    print(f"  ✓ {oligo['id']}: {stats['num_atoms']} atoms, avg degree: {stats['avg_degree']:.2f}")

# Convert to DataFrame for easier analysis
atom_stats_df = pd.DataFrame(atom_stats)
print(f"\nCalculated statistics for {len(atom_stats)} oligomers")

# Display the statistics table
print("\nAtom-level Network Statistics Summary:")
print("=" * 50)
display_cols = ['oligomer_id', 'num_atoms', 'num_bonds', 'avg_degree', 'max_degree', 
               'avg_clustering', 'diameter', 'carbon_count', 'oxygen_count']
print(atom_stats_df[display_cols].round(3))

In [ ]:
# Visualize atom-level network statistics
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. Degree distribution comparison
axes[0,0].bar(atom_stats_df['oligomer_id'], atom_stats_df['avg_degree'], 
             color='skyblue', alpha=0.7)
axes[0,0].set_title('Average Degree per Oligomer')
axes[0,0].set_ylabel('Average Degree')
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Clustering coefficient
axes[0,1].bar(atom_stats_df['oligomer_id'], atom_stats_df['avg_clustering'], 
             color='lightcoral', alpha=0.7)
axes[0,1].set_title('Average Clustering Coefficient')
axes[0,1].set_ylabel('Clustering Coefficient')
axes[0,1].tick_params(axis='x', rotation=45)

# 3. Graph density
axes[0,2].bar(atom_stats_df['oligomer_id'], atom_stats_df['density'], 
             color='lightgreen', alpha=0.7)
axes[0,2].set_title('Graph Density')
axes[0,2].set_ylabel('Density')
axes[0,2].tick_params(axis='x', rotation=45)

# 4. Element composition
x_pos = np.arange(len(atom_stats_df))
width = 0.25
axes[1,0].bar(x_pos - width, atom_stats_df['carbon_count'], width, 
             label='Carbon', color='#404040', alpha=0.7)
axes[1,0].bar(x_pos, atom_stats_df['oxygen_count'], width, 
             label='Oxygen', color='red', alpha=0.7)
axes[1,0].bar(x_pos + width, atom_stats_df['hydrogen_count'], width, 
             label='Hydrogen', color='lightblue', alpha=0.7)
axes[1,0].set_title('Element Composition')
axes[1,0].set_ylabel('Atom Count')
axes[1,0].set_xticks(x_pos)
axes[1,0].set_xticklabels(atom_stats_df['oligomer_id'], rotation=45)
axes[1,0].legend()

# 5. Diameter vs Average Path Length
axes[1,1].scatter(atom_stats_df['diameter'], atom_stats_df['avg_path_length'], 
                 c='purple', alpha=0.7, s=100)
axes[1,1].set_title('Diameter vs Avg Path Length')
axes[1,1].set_xlabel('Diameter')
axes[1,1].set_ylabel('Average Path Length')
for i, oligo_id in enumerate(atom_stats_df['oligomer_id']):
    axes[1,1].annotate(oligo_id, 
                      (atom_stats_df['diameter'].iloc[i], atom_stats_df['avg_path_length'].iloc[i]),
                      xytext=(5, 5), textcoords='offset points', fontsize=8)

# 6. Aromaticity analysis
axes[1,2].bar(atom_stats_df['oligomer_id'], atom_stats_df['aromaticity_ratio'], 
             color='orange', alpha=0.7)
axes[1,2].set_title('Aromaticity Ratio')
axes[1,2].set_ylabel('Aromatic Atoms / Total Atoms')
axes[1,2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.suptitle('Atom-level Network Statistics Analysis', fontsize=16, y=0.98)
plt.show()

print("\nDetailed statistical analysis plots generated successfully!")

## 6. Aggregated Network Statistics Across All Oligomers

Summarize and compare network properties across the entire dataset.

In [ ]:
# Calculate aggregated statistics
print("Aggregated Network Statistics Across All Oligomers")
print("=" * 55)

# Numerical columns for aggregation
numerical_cols = ['num_atoms', 'num_bonds', 'density', 'avg_degree', 'max_degree', 'min_degree',
                 'degree_std', 'avg_clustering', 'diameter', 'avg_path_length', 'radius',
                 'carbon_count', 'oxygen_count', 'hydrogen_count', 'aromatic_atoms', 
                 'aromatic_bonds', 'aromaticity_ratio']

# Calculate summary statistics
summary_stats = atom_stats_df[numerical_cols].describe()

print("Summary Statistics (Mean ± Std):")
print("-" * 40)
for col in numerical_cols:
    mean_val = summary_stats.loc['mean', col]
    std_val = summary_stats.loc['std', col]
    print(f"{col:20s}: {mean_val:8.3f} ± {std_val:6.3f}")

print("\n" + "=" * 55)
print("Key Findings:")
print(f"• Average oligomer size: {summary_stats.loc['mean', 'num_atoms']:.1f} atoms")
print(f"• Average connectivity: {summary_stats.loc['mean', 'avg_degree']:.2f} bonds per atom")
print(f"• Average clustering: {summary_stats.loc['mean', 'avg_clustering']:.3f}")
print(f"• Average aromaticity: {summary_stats.loc['mean', 'aromaticity_ratio']:.3f} ({summary_stats.loc['mean', 'aromaticity_ratio']*100:.1f}%)")
print(f"• C:O:H ratio: {summary_stats.loc['mean', 'carbon_count']:.1f}:{summary_stats.loc['mean', 'oxygen_count']:.1f}:{summary_stats.loc['mean', 'hydrogen_count']:.1f}")

# Create comprehensive comparison table
comparison_df = atom_stats_df[['oligomer_id'] + numerical_cols].copy()
print("\nDetailed Comparison Table:")
print(comparison_df.round(3))

In [ ]:
# Correlation analysis
print("Correlation Analysis Between Network Properties")
print("=" * 50)

# Calculate correlation matrix
correlation_cols = ['num_atoms', 'avg_degree', 'avg_clustering', 'density', 'diameter', 
                   'aromaticity_ratio', 'carbon_count', 'oxygen_count']
corr_matrix = atom_stats_df[correlation_cols].corr()

# Plot correlation heatmap
plt.figure(figsize=(10, 8))
im = plt.imshow(corr_matrix, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
plt.colorbar(im, shrink=0.8)

# Add correlation values to the plot
for i in range(len(correlation_cols)):
    for j in range(len(correlation_cols)):
        plt.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}', 
                ha='center', va='center', 
                color='white' if abs(corr_matrix.iloc[i, j]) > 0.5 else 'black')

plt.xticks(range(len(correlation_cols)), correlation_cols, rotation=45, ha='right')
plt.yticks(range(len(correlation_cols)), correlation_cols)
plt.title('Correlation Matrix of Network Properties')
plt.tight_layout()
plt.show()

# Identify strongest correlations
print("\nStrongest Correlations (|r| > 0.5):")
print("-" * 35)
for i in range(len(correlation_cols)):
    for j in range(i+1, len(correlation_cols)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.5:
            print(f"{correlation_cols[i]:15s} - {correlation_cols[j]:15s}: {corr_val:6.3f}")

## 7. Monomer-level Network Analysis

Analyze the oligomers at the monomer level using bond counts as proxy for inter-monomer connections. Since all oligomers have DP=3, we treat each as a network of 3 monomers connected by different linkage types.

In [ ]:
def create_monomer_network(bond_counts, oligomer_id, dp=3):
    """
    Create a monomer-level network based on bond counts.
    Assumes linear arrangement of monomers with inter-monomer bonds.
    
    Args:
        bond_counts (dict): Dictionary of bond types and counts
        oligomer_id (str): Identifier for the oligomer
        dp (int): Degree of polymerization (number of monomers)
        
    Returns:
        nx.Graph: Monomer-level network
    """
    G = nx.Graph()
    
    # Add monomer nodes
    for i in range(dp):
        G.add_node(f"M{i+1}", monomer_id=i+1, oligomer_id=oligomer_id)
    
    # Add edges based on bond counts
    # For DP=3, we typically have 2 inter-monomer connections
    edge_count = 0
    for bond_type, count in bond_counts.items():
        for _ in range(count):
            if edge_count < dp - 1:  # Don't exceed maximum possible connections
                start_monomer = edge_count
                end_monomer = edge_count + 1
                G.add_edge(f"M{start_monomer+1}", f"M{end_monomer+1}", 
                          bond_type=bond_type, weight=1)
                edge_count += 1
    
    return G

# Create monomer-level networks for all oligomers
print("Creating Monomer-level Networks")
print("=" * 35)

monomer_networks = []
monomer_stats = []

for oligo in oligomer_data:
    # Create monomer network
    mono_G = create_monomer_network(oligo['bond_counts'], oligo['id'])
    monomer_networks.append(mono_G)
    
    # Calculate basic statistics
    stats = {
        'oligomer_id': oligo['id'],
        'num_monomers': mono_G.number_of_nodes(),
        'num_connections': mono_G.number_of_edges(),
        'bond_types': list(oligo['bond_counts'].keys()),
        'bond_counts': oligo['bond_counts'],
        'is_linear': mono_G.number_of_edges() == mono_G.number_of_nodes() - 1,
        'avg_degree_mono': sum(dict(mono_G.degree()).values()) / mono_G.number_of_nodes() if mono_G.number_of_nodes() > 0 else 0
    }
    
    if nx.is_connected(mono_G) and mono_G.number_of_nodes() > 1:
        stats['diameter_mono'] = nx.diameter(mono_G)
        stats['avg_path_length_mono'] = nx.average_shortest_path_length(mono_G)
    else:
        stats['diameter_mono'] = 0
        stats['avg_path_length_mono'] = 0
    
    monomer_stats.append(stats)
    
    bond_types_str = ', '.join([f"{k}:{v}" for k, v in oligo['bond_counts'].items()])
    print(f"  {oligo['id']}: {stats['num_monomers']} monomers, {stats['num_connections']} connections [{bond_types_str}]")

# Convert to DataFrame
monomer_stats_df = pd.DataFrame(monomer_stats)
print(f"\nCreated monomer networks for {len(monomer_networks)} oligomers")

In [ ]:
# Visualize monomer-level networks
print("Monomer-level Network Visualizations")
print("=" * 40)

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

# Color mapping for different bond types
bond_type_colors = {
    'BB': 'red',       # Beta-beta
    'BO4': 'blue',     # Beta-O-4
    'B5': 'green',     # Beta-5
    '4O5': 'orange',   # 4-O-5
    '5O4': 'purple',   # 5-O-4
    'default': 'gray'
}

for i, (mono_G, stats) in enumerate(zip(monomer_networks, monomer_stats)):
    if i < 5:  # Display first 5 networks
        ax = axes[i]
        
        # Position nodes in a line for DP=3
        pos = {f"M{j+1}": (j, 0) for j in range(3)}
        
        # Draw nodes
        nx.draw_networkx_nodes(mono_G, pos, ax=ax, 
                              node_color='lightblue', 
                              node_size=1000,
                              alpha=0.8)
        
        # Draw edges with colors based on bond type
        edge_colors = []
        for u, v, data in mono_G.edges(data=True):
            bond_type = data.get('bond_type', 'default')
            color = bond_type_colors.get(bond_type, bond_type_colors['default'])
            edge_colors.append(color)
        
        nx.draw_networkx_edges(mono_G, pos, ax=ax,
                              edge_color=edge_colors,
                              width=3, alpha=0.7)
        
        # Add labels
        nx.draw_networkx_labels(mono_G, pos, ax=ax, 
                               font_size=10, font_weight='bold')
        
        # Add edge labels (bond types)
        edge_labels = {(u, v): data['bond_type'] for u, v, data in mono_G.edges(data=True)}
        nx.draw_networkx_edge_labels(mono_G, pos, edge_labels, ax=ax, font_size=8)
        
        ax.set_title(f"{stats['oligomer_id']}\nBonds: {', '.join([f'{k}:{v}' for k, v in stats['bond_counts'].items()])}")
        ax.set_xlim(-0.5, 2.5)
        ax.set_ylim(-0.5, 0.5)
        ax.axis('off')

# Add legend for bond types
if len(monomer_networks) < 6:
    ax = axes[5]
    ax.axis('off')
    
    # Create legend
    from matplotlib.lines import Line2D
    legend_elements = []
    for bond_type, color in bond_type_colors.items():
        if bond_type != 'default':
            legend_elements.append(Line2D([0], [0], color=color, lw=3, label=f'{bond_type} linkage'))
    
    ax.legend(handles=legend_elements, loc='center', fontsize=12)
    ax.set_title('Bond Type Legend')

plt.tight_layout()
plt.suptitle('Monomer-level Network Topology (DP=3)', fontsize=16, y=0.95)
plt.show()

print(f"\nDisplayed monomer networks for {min(5, len(monomer_networks))} oligomers")

In [ ]:
# Monomer-level statistics analysis
print("Monomer-level Network Analysis")
print("=" * 35)

# Bond type frequency analysis
all_bond_types = []
for stats in monomer_stats:
    for bond_type, count in stats['bond_counts'].items():
        all_bond_types.extend([bond_type] * count)

bond_type_freq = Counter(all_bond_types)
print("Bond Type Frequency:")
print("-" * 25)
for bond_type, count in bond_type_freq.most_common():
    percentage = (count / sum(bond_type_freq.values())) * 100
    print(f"{bond_type:8s}: {count:2d} ({percentage:5.1f}%)")

# Display monomer statistics table
print("\nMonomer-level Statistics:")
print("-" * 30)
display_df = monomer_stats_df[['oligomer_id', 'num_monomers', 'num_connections', 
                              'is_linear', 'avg_degree_mono', 'diameter_mono']].copy()
print(display_df.round(3))

# Create visualization of bond type distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Bond type frequency pie chart
bond_types = list(bond_type_freq.keys())
bond_counts = list(bond_type_freq.values())
colors = [bond_type_colors.get(bt, 'gray') for bt in bond_types]

ax1.pie(bond_counts, labels=bond_types, colors=colors, autopct='%1.1f%%',
        startangle=90, explode=[0.05]*len(bond_types))
ax1.set_title('Distribution of Inter-monomer Bond Types')

# Oligomer connectivity patterns
oligomer_ids = monomer_stats_df['oligomer_id']
num_connections = monomer_stats_df['num_connections']
colors_bar = ['lightblue' if linear else 'lightcoral' 
              for linear in monomer_stats_df['is_linear']]

bars = ax2.bar(oligomer_ids, num_connections, color=colors_bar, alpha=0.7)
ax2.set_title('Inter-monomer Connections per Oligomer')
ax2.set_ylabel('Number of Connections')
ax2.tick_params(axis='x', rotation=45)

# Add legend for bar colors
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='lightblue', label='Linear structure'),
    Patch(facecolor='lightcoral', label='Branched structure')
]
ax2.legend(handles=legend_elements)

plt.tight_layout()
plt.show()

print("\nMonomer-level Analysis Summary:")
print(f"• Average inter-monomer connections: {monomer_stats_df['num_connections'].mean():.2f}")
print(f"• Percentage of linear structures: {(monomer_stats_df['is_linear'].sum() / len(monomer_stats_df)) * 100:.1f}%")
print(f"• Most common bond type: {bond_type_freq.most_common(1)[0][0]} ({bond_type_freq.most_common(1)[0][1]} occurrences)")
print(f"• Average monomer degree: {monomer_stats_df['avg_degree_mono'].mean():.2f}")

## 8. Comprehensive Analysis Summary

Final summary of all findings from atom-level and monomer-level network analyses.

In [ ]:
print("=" * 70)
print("COMPREHENSIVE LIGNIN OLIGOMER NETWORK ANALYSIS SUMMARY")
print("=" * 70)

print("\n📊 DATASET OVERVIEW:")
print(f"   • Number of oligomers analyzed: {len(oligomer_data)}")
print(f"   • Degree of polymerization (DP): {data['dp']}")
print(f"   • Monomer type: {data['monoType']}")
print(f"   • Average molecular weight: {data['avgMolWt']:.1f} g/mol")

print("\n🔬 ATOM-LEVEL NETWORK ANALYSIS:")
print(f"   • Average oligomer size: {atom_stats_df['num_atoms'].mean():.1f} ± {atom_stats_df['num_atoms'].std():.1f} atoms")
print(f"   • Average bond count: {atom_stats_df['num_bonds'].mean():.1f} ± {atom_stats_df['num_bonds'].std():.1f} bonds")
print(f"   • Average degree: {atom_stats_df['avg_degree'].mean():.2f} ± {atom_stats_df['avg_degree'].std():.2f}")
print(f"   • Average clustering coefficient: {atom_stats_df['avg_clustering'].mean():.3f} ± {atom_stats_df['avg_clustering'].std():.3f}")
print(f"   • Average graph density: {atom_stats_df['density'].mean():.3f} ± {atom_stats_df['density'].std():.3f}")
print(f"   • Average diameter: {atom_stats_df['diameter'].mean():.1f} ± {atom_stats_df['diameter'].std():.1f}")

print("\n🧪 CHEMICAL COMPOSITION:")
avg_c = atom_stats_df['carbon_count'].mean()
avg_o = atom_stats_df['oxygen_count'].mean()
avg_h = atom_stats_df['hydrogen_count'].mean()
print(f"   • Average C:O:H ratio: {avg_c:.1f}:{avg_o:.1f}:{avg_h:.1f}")
print(f"   • Average aromaticity: {atom_stats_df['aromaticity_ratio'].mean():.3f} ({atom_stats_df['aromaticity_ratio'].mean()*100:.1f}%)")
print(f"   • Range of aromatic atoms: {atom_stats_df['aromatic_atoms'].min():.0f}-{atom_stats_df['aromatic_atoms'].max():.0f}")

print("\n🔗 MONOMER-LEVEL NETWORK ANALYSIS:")
print(f"   • All oligomers have: {monomer_stats_df['num_monomers'].iloc[0]} monomers (DP={data['dp']})")
print(f"   • Average inter-monomer connections: {monomer_stats_df['num_connections'].mean():.2f}")
print(f"   • Percentage with linear topology: {(monomer_stats_df['is_linear'].sum() / len(monomer_stats_df)) * 100:.1f}%")
print(f"   • Bond type distribution:")
for bond_type, count in bond_type_freq.most_common():
    percentage = (count / sum(bond_type_freq.values())) * 100
    print(f"     - {bond_type}: {count} bonds ({percentage:.1f}%)")

print("\n📈 KEY CORRELATIONS (|r| > 0.5):")
strong_corrs = []
for i in range(len(correlation_cols)):
    for j in range(i+1, len(correlation_cols)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.5:
            strong_corrs.append((correlation_cols[i], correlation_cols[j], corr_val))

if strong_corrs:
    for var1, var2, corr in strong_corrs[:5]:  # Show top 5
        print(f"   • {var1} ↔ {var2}: {corr:.3f}")
else:
    print("   • No strong correlations found (all |r| < 0.5)")

print("\n🎯 STRUCTURAL INSIGHTS:")
print(f"   • Network topology: {'Predominantly linear' if monomer_stats_df['is_linear'].mean() > 0.5 else 'Mixed linear/branched'}")
print(f"   • Molecular complexity: {'High' if atom_stats_df['avg_clustering'].mean() > 0.3 else 'Moderate' if atom_stats_df['avg_clustering'].mean() > 0.1 else 'Low'} clustering")
print(f"   • Size consistency: {'High' if atom_stats_df['num_atoms'].std() < 5 else 'Moderate' if atom_stats_df['num_atoms'].std() < 10 else 'Low'} (σ = {atom_stats_df['num_atoms'].std():.1f} atoms)")
print(f"   • Aromaticity consistency: {'High' if atom_stats_df['aromaticity_ratio'].std() < 0.05 else 'Moderate' if atom_stats_df['aromaticity_ratio'].std() < 0.1 else 'Variable'} (σ = {atom_stats_df['aromaticity_ratio'].std():.3f})")

print("\n" + "=" * 70)
print("Analysis completed successfully! All oligomers processed and analyzed.")
print("=" * 70)

## 9. Data Export and Reproducibility

Save the analysis results for further use and provide reproduction instructions.

In [ ]:
# Export analysis results to CSV files
print("Exporting Analysis Results...")
print("=" * 35)

# Export atom-level statistics
atom_stats_df.to_csv('lignin_atom_level_statistics.csv', index=False)
print("✓ Atom-level statistics saved to: lignin_atom_level_statistics.csv")

# Export monomer-level statistics
monomer_export_df = monomer_stats_df.copy()
# Convert bond_counts dict to string for CSV export
monomer_export_df['bond_counts_str'] = monomer_export_df['bond_counts'].apply(str)
monomer_export_df['bond_types_str'] = monomer_export_df['bond_types'].apply(lambda x: ','.join(x))
monomer_export_df.drop(['bond_counts', 'bond_types'], axis=1, inplace=True)
monomer_export_df.to_csv('lignin_monomer_level_statistics.csv', index=False)
print("✓ Monomer-level statistics saved to: lignin_monomer_level_statistics.csv")

# Export summary statistics
summary_stats.to_csv('lignin_summary_statistics.csv')
print("✓ Summary statistics saved to: lignin_summary_statistics.csv")

# Export correlation matrix
corr_matrix.to_csv('lignin_correlation_matrix.csv')
print("✓ Correlation matrix saved to: lignin_correlation_matrix.csv")

print("\n📋 FILES GENERATED:")
print(f"   • lignin_atom_level_statistics.csv ({len(atom_stats_df)} rows, {len(atom_stats_df.columns)} columns)")
print(f"   • lignin_monomer_level_statistics.csv ({len(monomer_export_df)} rows, {len(monomer_export_df.columns)} columns)")
print(f"   • lignin_summary_statistics.csv (statistical summaries)")
print(f"   • lignin_correlation_matrix.csv (correlation analysis)")

print("\n🔄 REPRODUCIBILITY NOTES:")
print("   • This analysis can be reproduced by running this notebook with the same dataset")
print("   • Required packages: pandas, matplotlib, networkx, rdkit, numpy")
print("   • Input data: LigninStructs_3.json (must be in same directory)")
print("   • All random seeds are fixed for consistent graph layouts")
print("   • Analysis assumes DP=3 for all oligomers as specified in the dataset")

print("\n✅ Analysis Complete!")
print("All molecular graphs visualized, statistics calculated, and results exported.")